In [1]:
# Import Libraries
import pandas as pd
from sqlalchemy import create_engine
import sys
import os
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
# load messages dataset
messages = pd.read_csv('disaster_messages.csv')
messages.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  
0  Un front froid se retrouve sur Cuba ce matin. ...  direct  
1                 Cyclone nan fini osinon li pa fini  direct  
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct  
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct  
4  facade ouest d Haiti et le reste du pays aujou...  direct

In [3]:
# load categories dataset
categories = pd.read_csv('disaster_categories.csv')
categories.head()

id                                         categories
0   2  related-1;request-0;offer-0;aid_related-0;medi...
1   7  related-1;request-0;offer-0;aid_related-1;medi...
2   8  related-1;request-0;offer-0;aid_related-0;medi...
3   9  related-1;request-1;offer-0;aid_related-1;medi...
4  12  related-1;request-0;offer-0;aid_related-0;medi...

### Merge Dataset

In [4]:
# merge datasets
df = pd.merge(messages, categories, on='id')
df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct   
1                 Cyclone nan fini osinon li pa fini  direct   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct   
4  facade ouest d Haiti et le reste du pays aujou...  direct   

                                          categories  
0  related-1;request-0;offer-0;aid_related-0;medi...  
1  related-1;request-0;offer-0;aid_related-1;medi...  
2  related-1;request-0;offer-0;aid_related-0;medi...  
3  related-1;request-1;offer-0;aid_related-1;medi...  
4  related-1;request-0;offer-0;aid_related-0;medi...

In [5]:
# create a dataframe

categories = df['categories'].str.split(';',expand=True)

In [6]:
categories.head()

0          1        2              3               4   \
0  related-1  request-0  offer-0  aid_related-0  medical_help-0   
1  related-1  request-0  offer-0  aid_related-1  medical_help-0   
2  related-1  request-0  offer-0  aid_related-0  medical_help-0   
3  related-1  request-1  offer-0  aid_related-1  medical_help-0   
4  related-1  request-0  offer-0  aid_related-0  medical_help-0   

                   5                    6           7           8   \
0  medical_products-0  search_and_rescue-0  security-0  military-0   
1  medical_products-0  search_and_rescue-0  security-0  military-0   
2  medical_products-0  search_and_rescue-0  security-0  military-0   
3  medical_products-1  search_and_rescue-0  security-0  military-0   
4  medical_products-0  search_and_rescue-0  security-0  military-0   

              9   ...             26                      27  \
0  child_alone-0  ...  aid_centers-0  other_infrastructure-0   
1  child_alone-0  ...  aid_centers-0  other_infrastructure-0   
2  child_alone-0  ...  aid_centers-0  other_infrastructure-0   
3  child_alone-0  ...  aid_centers-0  other_infrastructure-0   
4  child_alone-0  ...  aid_centers-0  other_infrastructure-0   

                  28        29       30      31            32      33  \
0  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
1  weather_related-1  floods-0  storm-1  fire-0  earthquake-0  cold-0   
2  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
3  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
4  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   

                34               35  
0  other_weather-0  direct_report-0  
1  other_weather-0  direct_report-0  
2  other_weather-0  direct_report-0  
3  other_weather-0  direct_report-0  
4  other_weather-0  direct_report-0  

[5 rows x 36 columns]

In [7]:
row = categories.iloc[0,:].unique

category_colnames = [i[:-2] for i in categories.loc[0,:].unique()]

print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [8]:
categories.columns = category_colnames
categories.head()

related    request    offer    aid_related    medical_help  \
0  related-1  request-0  offer-0  aid_related-0  medical_help-0   
1  related-1  request-0  offer-0  aid_related-1  medical_help-0   
2  related-1  request-0  offer-0  aid_related-0  medical_help-0   
3  related-1  request-1  offer-0  aid_related-1  medical_help-0   
4  related-1  request-0  offer-0  aid_related-0  medical_help-0   

     medical_products    search_and_rescue    security    military  \
0  medical_products-0  search_and_rescue-0  security-0  military-0   
1  medical_products-0  search_and_rescue-0  security-0  military-0   
2  medical_products-0  search_and_rescue-0  security-0  military-0   
3  medical_products-1  search_and_rescue-0  security-0  military-0   
4  medical_products-0  search_and_rescue-0  security-0  military-0   

     child_alone  ...    aid_centers    other_infrastructure  \
0  child_alone-0  ...  aid_centers-0  other_infrastructure-0   
1  child_alone-0  ...  aid_centers-0  other_infrastructure-0   
2  child_alone-0  ...  aid_centers-0  other_infrastructure-0   
3  child_alone-0  ...  aid_centers-0  other_infrastructure-0   
4  child_alone-0  ...  aid_centers-0  other_infrastructure-0   

     weather_related    floods    storm    fire    earthquake    cold  \
0  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
1  weather_related-1  floods-0  storm-1  fire-0  earthquake-0  cold-0   
2  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
3  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   
4  weather_related-0  floods-0  storm-0  fire-0  earthquake-0  cold-0   

     other_weather    direct_report  
0  other_weather-0  direct_report-0  
1  other_weather-0  direct_report-0  
2  other_weather-0  direct_report-0  
3  other_weather-0  direct_report-0  
4  other_weather-0  direct_report-0  

[5 rows x 36 columns]

In [9]:
for column in categories:
    
    categories[column] = categories[column].apply(lambda x:x.split('-')[-1]).astype(int)

categories.head()

related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone  ...  aid_centers  \
0                  0         0         0            0  ...            0   
1                  0         0         0            0  ...            0   
2                  0         0         0            0  ...            0   
3                  0         0         0            0  ...            0   
4                  0         0         0            0  ...            0   

   other_infrastructure  weather_related  floods  storm  fire  earthquake  \
0                     0                0       0      0     0           0   
1                     0                1       0      1     0           0   
2                     0                0       0      0     0           0   
3                     0                0       0      0     0           0   
4                     0                0       0      0     0           0   

   cold  other_weather  direct_report  
0     0              0              0  
1     0              0              0  
2     0              0              0  
3     0              0              0  
4     0              0              0  

[5 rows x 36 columns]

In [10]:
df.drop('categories',axis=1, inplace=True)

df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  
0  Un front froid se retrouve sur Cuba ce matin. ...  direct  
1                 Cyclone nan fini osinon li pa fini  direct  
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct  
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct  
4  facade ouest d Haiti et le reste du pays aujou...  direct

In [11]:
categories.head()

related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone  ...  aid_centers  \
0                  0         0         0            0  ...            0   
1                  0         0         0            0  ...            0   
2                  0         0         0            0  ...            0   
3                  0         0         0            0  ...            0   
4                  0         0         0            0  ...            0   

   other_infrastructure  weather_related  floods  storm  fire  earthquake  \
0                     0                0       0      0     0           0   
1                     0                1       0      1     0           0   
2                     0                0       0      0     0           0   
3                     0                0       0      0     0           0   
4                     0                0       0      0     0           0   

   cold  other_weather  direct_report  
0     0              0              0  
1     0              0              0  
2     0              0              0  
3     0              0              0  
4     0              0              0  

[5 rows x 36 columns]

In [12]:
df = pd.concat([df, categories], axis=1)
df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products  ...  \
0        0      0            0             0                 0  ...   
1        0      0            1             0                 0  ...   
2        0      0            0             0                 0  ...   
3        1      0            1             0                 1  ...   
4        0      0            0             0                 0  ...   

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0                0       0      0     0   
1            0                     0                1       0      1     0   
2            0                     0                0       0      0     0   
3            0                     0                0       0      0     0   
4            0                     0                0       0      0     0   

   earthquake  cold  other_weather  direct_report  
0           0     0              0              0  
1           0     0              0              0  
2           0     0              0              0  
3           0     0              0              0  
4           0     0              0              0  

[5 rows x 40 columns]

In [13]:
#check the number of duplicates

df.duplicated().sum()

170

In [14]:
# drop duplicates
df = df.drop_duplicates()

In [15]:
df.head()

id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products  ...  \
0        0      0            0             0                 0  ...   
1        0      0            1             0                 0  ...   
2        0      0            0             0                 0  ...   
3        1      0            1             0                 1  ...   
4        0      0            0             0                 0  ...   

   aid_centers  other_infrastructure  weather_related  floods  storm  fire  \
0            0                     0                0       0      0     0   
1            0                     0                1       0      1     0   
2            0                     0                0       0      0     0   
3            0                     0                0       0      0     0   
4            0                     0                0       0      0     0   

   earthquake  cold  other_weather  direct_report  
0           0     0              0              0  
1           0     0              0              0  
2           0     0              0              0  
3           0     0              0              0  
4           0     0              0              0  

[5 rows x 40 columns]

## MACHINE LEARNING PIPELINE PREPARATION

In [16]:
# extract values from X nad Y
X = df['message'].values
Y = df.iloc[:, 4:].values
category_names = list(df.columns[4:])

In [17]:
def tokenize(text):
    """
    input:
        text: Message data for tokenization.
    output:
        clean_tokens: Result list after tokenization.
    """
    text = re.sub(r'[^A-Za-z0-9]'," ",text.lower())
    text = word_tokenize(text)
    words = [w.strip() for w in text if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return clean_tokens

In [18]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,f1_score,precision_recall_curve,accuracy_score
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline = Pipeline([('CountVect',CountVectorizer(tokenizer=tokenize)),
                     ('Tfidf',TfidfTransformer()),
                     ('MultiClass',MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))])

pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


In [20]:
for i in range(y_test.shape[1]):
    print('{} accuracy: {}'.format(category_colnames[i],accuracy_score(y_test[:,i], y_pred[:,i])))

related accuracy: 0.8220933780897162
request accuracy: 0.897467195605737
offer accuracy: 0.9949649069270674
aid_related accuracy: 0.7610619469026548
medical_help accuracy: 0.9298138541348795
medical_products accuracy: 0.9584986267927983
search_and_rescue accuracy: 0.9748245346353372
security accuracy: 0.9819957277998169
military accuracy: 0.9716203844980165
child_alone accuracy: 1.0
water accuracy: 0.9620079340860543
food accuracy: 0.9484284406469332
shelter accuracy: 0.9478181263350626
clothing accuracy: 0.9859627708269759
money accuracy: 0.977570949038755
missing_people accuracy: 0.9893194995422643
refugees accuracy: 0.9705523344522429
death accuracy: 0.9705523344522429
other_aid accuracy: 0.8657308513884651
infrastructure_related accuracy: 0.9330180042722002
transport accuracy: 0.9594140982606042
buildings accuracy: 0.9604821483063778
electricity accuracy: 0.981080256332011
tools accuracy: 0.9946597497711321
hospitals accuracy: 0.9909978638999084
shops accuracy: 0.9951174855050351
a

In [21]:
pipeline.get_params()

{'memory': None,
 'steps': [('CountVect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x00000209A5E483A8>,
                   vocabulary=None)),
  ('Tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('MultiClass',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                                           class_weight=None,
                                                                           dual=True,
                                                                           fit_intercept=Tru

In [23]:
parameters = {'CountVect__binary': [False,True],
              'Tfidf__norm': ['l1','l2'],
             'MultiClass__estimator__estimator__C': [1.0,3.0]
             }

cv = GridSearchCV(pipeline,param_grid=parameters)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label not 0 is pre